In [1]:
# Parameters 

region_name = 'NNI'
var_name = 'TMEAN'
target_type = 'cat3_categories'
GCM = 'ECMWF'
# GCM = 'All'
standardized = False

In [2]:
%matplotlib inline

In [3]:
import os 
import sys 
import pathlib

In [4]:
HOME = pathlib.Path.home()

In [5]:
from matplotlib import pyplot as plt

In [6]:
import numpy as np 
import pandas as pd

In [7]:
from pycaret import datasets

In [8]:
from pycaret.classification import *

In [9]:
dpath = HOME / 'research' / 'Smart_Ideas' / 'outputs' / 'CSVs'

In [10]:
list(dpath.glob("*.csv"))

[PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_RAIN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_TMEAN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_TMEAN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_RAIN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_RAIN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_TMEAN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_TMEAN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_RAIN_training_set.csv')]

In [11]:
if standardized: 
    train_data = pd.read_csv(dpath / f'GCMs_std_and_targets_cat3_and_anomalies_{var_name}_training_set.csv', index_col=0, parse_dates=True) 
    test_data = pd.read_csv(dpath / f'GCMs_std_and_targets_cat3_and_anomalies_{var_name}_test_set.csv', index_col=0, parse_dates=True)
else: 
    train_data = pd.read_csv(dpath / f'GCMs_and_targets_cat3_and_anomalies_{var_name}_training_set.csv', index_col=0, parse_dates=True) 
    test_data = pd.read_csv(dpath / f'GCMs_and_targets_cat3_and_anomalies_{var_name}_test_set.csv', index_col=0, parse_dates=True)    

### small function that prepares the data for ingestion in `PyCARET` 

In [12]:
# %%writefile ../../../../ml4seas/GCM/prepare_data_CSV_to_CARET.py
def prepare_data_CSV_to_CARET(data, gcm_index=-13, GCM='All', region_name=None, target_type=None, scaling=True, doPCA=True, n_components=0.9): 
    """
    Small function that prepares the data initially contained in the 
    processed CSV files for ingestion into PyCARET
    
    Arguments
    --------- 
    
    - gcm_index : negative index (always) indicating what 
            is the last column containing the predictor variable 
            default is -12 for the 'ext_regional' geographical 
            domain 
    - GCM: 'All' or specific GCM in ['CMCC', 'CanCM4i', 'CanSIPSv2', 
                                    'DWD', 'ECMWF', 'GEM_NEMO', 'JMA',
                                    'METEO_FRANCE', 'NASA_GEOSS2S', 'NCEP_CFSv2', 'UKMO']
    - region_name : ['NNI','WNI','ENI','NSI','WSI','ESI']
    - target_type : 'cat3_category' or 'anomalies'
    
    Return
    ------
    
    - data : the data filtered by GCM (if not 'All'), region_name and target type 
    - GCM_index: the GCMs index 

    """
    
    import numpy as np 
    import pandas as pd 
    from sklearn.preprocessing import  StandardScaler 
    from sklearn.decomposition import PCA

    # extract one GCM if not 'All'
    if GCM != 'All': 
        data = data.query(f"GCM == '{GCM}'") 
    
    # GCM name and associated index
    GCMs_name = data.loc[:,['GCM']]  
    
    # GCM (features)
    GCM_data = data.iloc[:,0:gcm_index]
    
    # associated index
    index = GCM_data.index
    
    # associated column names 
    cols = GCM_data.columns

    # target variable 
    target = data.loc[:,[f'{region_name}_{target_type}']]
    
    # get the values for X 
    X = GCM_data.values
    
    if scaling:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    else: 
        scaler = None
    
    if doPCA: 
        pca = PCA(n_components=n_components)
        pca = pca.fit(X)
        X = pca.transform(X)
        npcs = X.shape[1]
    else: 
        pca = None

    # now casts X into a DataFrame
    if doPCA: 
        df = pd.DataFrame(X, index=index, columns=[f"PC{i}" for i in range(1, npcs+1)])
    else: 
        df = pd.DataFrame(GCM_data.values, index=index, columns=cols)
        
    # add the target variable 
    data = pd.concat([df, target], axis=1)
    
    return data, GCMs_name, scaler, pca

In [13]:
train_data, GCMs_name_train, scaler, pca = prepare_data_CSV_to_CARET(train_data, GCM=GCM, region_name=region_name, target_type=target_type, scaling=True, doPCA=True)

In [14]:
train_data.shape

(288, 34)

In [15]:
train_data.columns

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28',
       'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'NNI_cat3_categories'],
      dtype='object')

In [16]:
test_data, GCMs_name_test, _, _ = prepare_data_CSV_to_CARET(test_data, GCM=GCM, region_name=region_name, target_type=target_type, scaling=False, doPCA=False)

In [17]:
test_data.shape

(33, 4930)

In [19]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
pca

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

### set up the PYCARET experiment, use only the training set (cross validation will be used to evaluate the models)

### Note that the first time around, the argument `silent` is left to the default (False) so that the user can verify the data type of each column after the first run and if data types correctly interpreted, `silent` can be set to True

In [ ]:
silent = True

In [ ]:
exp_clf = setup(data = train_data, target = f'{region_name}_{target_type}', session_id=123,
                  normalize = False, 
                  transformation = False, 
                  pca=False, silent=silent)

### quick comparison of the models 

In [ ]:
df_compare_models = compare_models()

In [ ]:
df_compare_models.data

In [ ]:
df_compare_models.data.loc[[0],:]

In [ ]:
lg = create_model('lightgbm')

### first pathway: tune the best model 

In [ ]:
tuned_best_model = tune_model('lightgbm')

In [ ]:
test_data.shape

In [ ]:
test_data.head()

In [ ]:
X = scaler.transform(test_data)

### doesnt lead to boost in accuracy 

### second approach: blend all models 

In [ ]:
blend_all = blend_models()

### doesnt lead to boost in accuracy either 

### third approach: blend the 3 best models (note that cannot use catboost)

In [ ]:
df_compare_models.data.loc[0:5,:]

### first step: create models 

In [ ]:
lg = create_model('lightgbm')

In [ ]:
et = create_model('et')

In [ ]:
xg = create_model('xgboost')

In [ ]:
blend_specific = blend_models(estimator_list = [et,xg, lg])

### no improvement 

In [ ]:
test_data.columns

In [ ]:
blend_specific.predict?